<a href="https://colab.research.google.com/github/AchrafAsh/deepfakes/blob/main/deepfake_vtoonify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training VToonify

### 1. Train DualStyleGAN

In [ ]:
!git clone https://github.com/AchrafAsh/DualStyleGAN

Cloning into 'DualStyleGAN'...
remote: Enumerating objects: 767, done.
remote: Counting objects: 100% (260/260), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 767 (delta 214), reused 193 (delta 188), pack-reused 507
Receiving objects: 100% (767/767), 26.24 MiB | 18.66 MiB/s, done.
Resolving deltas: 100% (382/382), done.


In [ ]:
import os
os.chdir("DualStyleGAN")

Download the models to `./checkpoint/`

- `stylegan2-ffhq-config-f.pt`: StyleGAN model trained on FFHQ taken from rosinality
- `model_ir_se50.pth`: Pretrained IR-SE50 model taken from TreB1eN for ID loss

Prepare a dataset of the style to learn (300 images of Tom Cruise) in ./data/cruise/images/train

In [ ]:
!unzip ./data/cruise.zip -d ./data/
!rm -r ./data/__MACOSX

In [ ]:
!mkdir ./data/cruise/images/train
!mv ./data/cruise/images/*.png ./data/cruise/images/train

In [ ]:
!pip install lmdb

In [ ]:
!python ./model/stylegan/prepare_data.py --out ./data/cruise/lmdb \
    --n_worker 4 --size 1024 ./data/cruise/images

Make dataset of image sizes: 1024
0it [00:00, ?it/s]/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional.py:442: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional.py:442: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional.py:442: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional.py:442: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [ ]:
!pip install ninja

Fine tune StyleGAN:

*The fine-tuned model can be found in* ./checkpoint/cartoon/finetune-000600.pt. *Intermediate results are saved in* ./log/cartoon/

In [ ]:
!python -m torch.distributed.launch --nproc_per_node=8 \
    finetune_stylegan.py --iter 600 --batch 4 --local_rank 0\
    --ckpt ./checkpoint/stylegan2-ffhq-config-f.pt --style cruise \
    --augment ./data/cruise/lmdb/

/usr/local/lib/python3.9/dist-packages/torch/distributed/launch.py:180: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Load options
ada_every: 256
ada_length: 500000
ada_target: 0.6
augment: True
augment_p: 0
batch: 4
channel_multiplier: 2
ckpt: ./checkpoint/stylegan2-ffhq-config-f.pt
d_reg_every: 16
g_reg_every: 4
iter: 600
local_rank: 0
lr: 0.002
mixi

In [ ]:
!python finetune_stylegan.py --iter 600 --batch 4\
    --ckpt ./checkpoint/stylegan2-ffhq-config-f.pt \
    --style cruise --augment ./data/cruise/lmdb/

Load options
ada_every: 256
ada_length: 500000
ada_target: 0.6
augment: True
augment_p: 0
batch: 4
channel_multiplier: 2
ckpt: ./checkpoint/stylegan2-ffhq-config-f.pt
d_reg_every: 16
g_reg_every: 4
iter: 600
local_rank: 0
lr: 0.002
mixing: 0.9
model_path: ./checkpoint/
n_sample: 9
path: ./data/cruise/lmdb/
path_batch_shrink: 2
path_regularize: 2
r1: 10
save_every: 10000
size: 1024
style: cruise
wandb: False
**************************************************************************************************
load model: ./checkpoint/stylegan2-ffhq-config-f.pt
  0%|                                                                                                               | 0/600 [00:00<?, ?it/s]/content/DualStyleGAN/model/stylegan/op/conv2d_gradfix.py:88: UserWarning: conv2d_gradfix not supported on PyTorch 1.13.1+cu116. Falling back to torch.nn.functional.conv2d().
  warnings.warn(
  0%|                                                                                                     

Destylize artistic portraits:

In [ ]:
!python destylize.py --model_name finetune-000600.pt --batch 1 --iter 300 cruise

Progressive fine tuning:

In [ ]:
!python -m torch.distributed.launch --nproc_per_node=1 \
    --master_port=8765 pretrain_dualstylegan.py --iter 3000 \
    --batch 4 ./data/ffhq/lmdb/

Fine-tune on target domain

In [ ]:
!python -m torch.distributed.launch --nproc_per_node=8 \
    --master_port=8765 finetune_dualstylegan.py --iter 1500 \
    --batch 4 --ckpt ./checkpoint/generator-pretrain.pt \
    --style_loss 0.25 --CX_loss 0.25 --perc_loss 1 --id_loss 1 \
    --L2_reg_loss 0.015 --augment cruise

### 2. Training VToonify

Download models checkpoints in `./checkpoint/`:
- `StyleGAN` for improving image quality
- `Pixel2style2pixel` encoder that embeds FFHQ images into StyleGAN2 Z+ latent code
- `BiSeNet` for face parsing from face-parsing.PyTorch
- Editing vectors taken from `LowRankGAN` for editing face attributes
- Pre-trained stylegan-based toonification models from Part 1 above

In [ ]:
# for pre-training the encoder
!python -m torch.distributed.launch --nproc_per_node=N_GPU --master_port=PORT train_vtoonify_d.py \
       --iter ITERATIONS --stylegan_path DUALSTYLEGAN_PATH --exstyle_path EXSTYLE_CODE_PATH \
       --batch BATCH_SIZE --name SAVE_NAME --pretrain

In [ ]:
# for training VToonify-D given the pre-trained encoder
!python -m torch.distributed.launch --nproc_per_node=N_GPU --master_port=PORT train_vtoonify_d.py \
       --iter ITERATIONS --stylegan_path DUALSTYLEGAN_PATH --exstyle_path EXSTYLE_CODE_PATH \
       --batch BATCH_SIZE --name SAVE_NAME                  # + ADDITIONAL STYLE CONTROL OPTIONS

See example here:
https://github.com/williamyang1991/VToonify#:~:text=Here%20is%20an%20example%20to%20reproduce%20the%20VToonify%2DDsd%20on%20Cartoon%20style%20and%20the%20VToonify%2DD%20specialized%20for%20a%20mild%20toonification%20on%20the%2026th%20cartoon%20style%3A